## Notbook E: Time-resolved reconstruction

#### Prerequisites:
- a simulated rawdata file containing time-varying magnetisation.

#### Goals:
- reconstruct time-resolved images containing the MRF signal.

#### Content overview: 
- setting up time-resolved coilmaps for reconstruction
- reconstructing the data using a pseudo-inverse 

In [ ]:
from pathlib import Path
import os 
import auxiliary_functions as aux
import time
import numpy as np 
import sirf.Gadgetron as pMR
import matplotlib.pyplot as plt

# this is where we store the properly formatted data
root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"
fpath_input = root_path / "Output"

In [ ]:
# load the simulated mrf acquisition
fname_simulated_file = fpath_input / "output_d_simulate_mrf_static_1500.h5"
ad = pMR.AcquisitionData(str(fname_simulated_file))

In [ ]:
# first compute the CSM based on all data
csm = pMR.CoilSensitivityData()
csm.smoothness = 50
csm.calculate(ad)
csm_arr = csm.as_array()

print(csm_arr.shape)


In [ ]:

# then activate the time-resolved reconstruction in repetition dimension
num_recon_imgs = 250
ad = aux.activate_timeresolved_reconstruction(ad, num_recon_imgs)

# set up a new CSM based on the time-resolved acquisition data
csm = pMR.CoilSensitivityData()
# this step sets up a time-resolved coilmap. The reconstruction checks if for each
# time point a coilmap is present. 
csm.calculate(ad) 

In [ ]:

# But we want to use the coilmap that was computed from the entire dataset
# so we give every repetition the same coilmap
num_reps = csm.as_array().shape[1]
print(num_reps)
csm_arr = np.tile(csm_arr, (1,num_reps,1,1))

# unfortunately these two axes have to be swapped.
csm_arr = np.swapaxes(csm_arr, 0, 1)
csm = csm.fill(csm_arr.astype(csm.as_array().dtype))

In [ ]:

tstart = time.time()
recon = aux.reconstruct_data(ad, csm)
print("--- Required {} seconds for reconstruction.".format( time.time()-tstart))

import nibabel as nib
img = nib.Nifti1Image(np.abs(recon.as_array()), np.eye(4))
nib.save(img,"/media/sf_CCPPETMR/tmp_mrfresolved_nacq_{}.nii".format(num_recon_imgs))

This took quite some time to reconstrut so we store this time-resolved data and reconstruction so we can quickly access them for the next notebook

In [ ]:

fname_timeresolved_ad = fpath_input / "output_e_timeresolved_mrf_simulation.h5"
ad.write(str(fname_timeresolved_ad))

fname_timeresolved_recon = fpath_input / "output_e_timeresolved_recon_mrf_simulation.npy"
np.save(fname_timeresolved_recon, recon.as_array())

### Recap
In this notebook we:
- set up time-resolved acquisition data
- constructed a time-resolved coilmap from the entire dataset
- performed a time-resolved reconstruction of the MRF dataset.

__Up next: dictionary-matching to create T1 and T2 maps.__